In [1]:
import pandas as pd
import json
import numpy as np
import pymysql
import datetime
import time
import os
import h5py
import math
import collections
import glob
import matplotlib.pyplot as plt
from obspy.clients.iris import Client
from tqdm import tqdm
from scipy.signal import butter, filtfilt
from scipy import integrate
from scipy.signal import sosfilt, iirfilter, zpk2sos
from multiprocessing import Process, Manager

In [45]:
%run utils.ipynb

In [128]:
def parse_file(p, s, h5path, station_list, sensor, usage, record_inst):
    all_df = pd.DataFrame()
    df = pd.DataFrame()
    
    client = Client()
    
    hf = h5py.File(h5path, 'a')
    
    # source attributes
    df['source_origin_time'] = [p['ori_time']]
    df['source_depth_km'] = [p['depth']]
    df['source_latitude_deg'] = [p['lat']]
    df['source_longitude_deg'] = [p['lon']]
    df['source_magnitude'] = [p['mag']]
    df['source_magnitude_type'] = ['ml']    # 芮氏規模
    
#     for k in tqdm(p.keys(), total=len(list(p.keys()))):
    for k in p.keys():
        try:
            if 'distance' in p[k].keys():
                df['station_code'] = [k]

                # 新格式的 network = TW
                df['station_network_code'] = ['TW']
                
                df['station_p_polar'] = p[k]['p_polar']
                df['trace_channel'] = ['HL']
                df['trace_start_time'] = [p[k]['starttime']]

                _, lon, lat, ele, location = get_factor('HWA047', -1, -1, -1, df['trace_start_time'].iloc[0], s, station_list, sensor, usage, 'TSMIP')
                df['station_longitude_deg'], df['station_latitude_deg'], df['station_elevation_m'] = [lon], [lat], [ele]
                df['station_location_code'] = [str(location)]
                df['trace_factor_ZNE'] = [[1,1,1]]
                
                # 若有多的事件在同個波型，則對應到相同的 trace
                tmp_trace_name = p[k]['waveFile'].split('.')[:2]
                tmp_trace_name = "_".join(tmp_trace_name).replace('/', '_')[1:]
                df['trace_name'] = [tmp_trace_name + '_' + df['station_location_code'].iloc[0]]
                
                sampling_rate = p[k]['sampling_rate']
                df['trace_sampling_rate_hz'] = [sampling_rate]

                df['trace_npts'] = [int(sampling_rate*p[k]['RecordLength'])]
                
                # p-wave
                df['trace_p_arrival_sample'] = [time_diff_sample(p[k]['starttime'], p[k]['p_arrival_time'], sampling_rate)]
                df['trace_p_status'] = ['maunal']
                df['trace_p_weight'] = [p[k]['p_weight']]
                
                # s-wave
                df['trace_s_arrival_sample'] = [time_diff_sample(p[k]['starttime'], p[k]['s_arrival_time'], sampling_rate)]
                df['path_p_travel_s'] = [(df['trace_s_arrival_sample'].iloc[0]-df['trace_p_arrival_sample'].iloc[0]) / sampling_rate ]
                df['trace_s_status'] = ['maunal']
                df['trace_s_weight'] = [p[k]['s_weight']]

                df['path_ep_distance_km'] = [p[k]['distance']]
                df['path_back_azimuth_deg'] = [client.distaz(stalat=df['station_latitude_deg'].iloc[0], stalon=df['station_longitude_deg'].iloc[0], 
                                                            evtlat=df['source_latitude_deg'].iloc[0], evtlon=df['source_longitude_deg'].iloc[0])['backazimuth']]

                df['trace_category'] = ['earthquake']

                # 檢查 waveform 可不可用
                wave_z, wave_n, wave_e = readWave_TSMIP('/mnt/nas2/ori_TSMIP/event_info/wave'+p[k]['waveFile'])

                df['trace_snr_db'] = [round(snr_in_db(wave_z, wave_n, wave_e, df['trace_p_arrival_sample'].iloc[0]), 3)]
                df['trace_Z_snr_db'] = [round(Zsnr_in_db(wave_z, df['trace_p_arrival_sample'].iloc[0]), 3)]

                pga, pga_g, z_pga = calc_pga(wave_z, wave_n, wave_e, sampling_rate, 'FBA')
                pgv, z_pgv = calc_pgv(wave_z, wave_n, wave_e, sampling_rate, 'FBA')

                df['trace_pga_perg'] = [round(pga_g, 3)]
                df['trace_pga_cmps2'] = [round(pga, 3)]
                df['trace_Z_pga_cmps2'] = [round(z_pga, 3)]
                df['trace_pgv_cmps'] = [round(pgv, 3)]
                df['trace_Z_pgv_cmps'] = [round(z_pgv, 3)]

                if p[k]['Z'] == 1 and p[k]['N'] == 1 and p[k]['E'] == 1 and (p[k]['time_system'] == 1 or p[k]['time_system'] == 0):
                    df['trace_completeness'] = [1]
                else:
                    df['trace_completeness'] = [0]
                    
                df['trace_number_of_event'] = [1]
                    
                all_df = pd.concat([all_df, df])

                # write trace into hdf5 file: (3, npts)
                wave = np.vstack((wave_z, wave_n, wave_e))

                hf.create_dataset('data/'+df['trace_name'].iloc[0], wave.shape, data=wave)
        except Exception as exx:
#             print(exx)
            pass

    return all_df

In [129]:
def getfile(base_dir, year):
    files = glob.glob(base_dir+year+'/*.json')
    
    return files

In [6]:
base_dir = "/mnt/nas2/TSMIP/"

s = get_StationInfo()
station_list = pd.read_excel("/mnt/nas4/weiwei/seisbench_CWBSN/Station.xlsx")
sensor = pd.read_excel("/mnt/nas4/weiwei/seisbench_CWBSN/Sensor.xlsx")
usage = pd.read_excel("/mnt/nas4/weiwei/seisbench_CWBSN/Usage.xlsx")
record_inst = 'TSMIP'

format = {
    'dimension_order': 'CW',
    'component_order': 'ZNE',
    'sampling_rate': 200,
    'unit': 'cmps2',
    'measurement': 'acceleration'
}

if not os.path.exists("/mnt/nas2/TSMIP/seisbench"):
    os.makedirs("/mnt/nas2/TSMIP/seisbench")

In [ ]:
def main(y):
    df = pd.DataFrame()
    print('year: ', y)
    
    files = getfile(base_dir, y)
    
    for f in tqdm(files, total=len(files)):    
        json_file = open(f)
        p = json.load(json_file)
        
        json_df = parse_file(p, s, '/mnt/nas2/TSMIP/seisbench/chunks_' + y + '.hdf5',
                            station_list, sensor, usage, record_inst)
         
        df = pd.concat([df, json_df])
        
    # 計算每個 trace 中有多少事件
    df = update_multiple_event(df)
    
    # write into files: metadata
    df.to_csv('/mnt/nas2/TSMIP/seisbench/metadata_' + y + '.csv', index=False)
    
    # write into hdf5: chunks/data_format
    hf = h5py.File('/mnt/nas2/TSMIP/seisbench/chunks_' + y + '.hdf5', 'a')
    for k, v in format.items():
        hf.create_dataset('data_format/'+k, data=v)

In [132]:
main('2020')

year:  2020


  7%|██████████▊                                                                                                                                              | 121/1705 [02:15<16:05,  1.64it/s]/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versi

/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versio

In [133]:
years = [str(y) for y in range(2011, 2021)]
years.remove('2020')

for i in years:
    p = Process(target=main, args=([str(i)]))
    p.start()

year:  2011
year: 

  0%|                                                                                                                                                                    | 0/848 [00:00<?, ?it/s]

year:  2012 2013
year: 
 year: 2014 
2015year: 
 2016
year:  

  0%|                                                                                                                                                                   | 0/1214 [00:00<?, ?it/s]

2017

...

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                 | 2380/3560 [1:30:25<13:51,  1.42it/s]/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 67%|████████████████████████

/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versio

/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/v

/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/v

/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/nas4/weiwei/.pyenv/versions/earthquake/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/nas4/weiwei/.pyenv/versio

In [56]:
data = h5py.File("/mnt/nas2/TSMIP/seisbench/chunks_2019.hdf5", 'r')
data.keys()

<KeysViewHDF5 ['data', 'data_format']>

In [55]:
df = pd.read_csv("/mnt/nas2/CWBSN_modify_time/seisbench/metadata_2020.csv")
df.head()

,source_origin_time,source_depth_km,source_latitude_deg,source_longitude_deg,source_magnitude,source_magnitude_type,source_gap_deg,station_code,station_network_code,station_location_code,...,trace_category,trace_snr_db,trace_Z_snr_db,trace_pga_perg,trace_pga_cmps2,trace_Z_pga_cmps2,trace_pgv_cmps,trace_Z_pgv_cmps,trace_number_of_event,trace_completeness
0,2020-12-27 15:39:16.359999,34.17,22.09,121.32,3.94,ml,152,LAY,TW,10,...,earthquake,16.029,8.936,0.556,5.455,2.292,4.483,4.713,1,1
1,2020-12-27 15:39:16.359999,34.17,22.09,121.32,3.94,ml,152,LAY,TW,10,...,earthquake,26.398,30.422,0.423,4.153,1.934,0.104,0.056,1,1
2,2020-12-27 15:39:16.359999,34.17,22.09,121.32,3.94,ml,152,LAY,TW,0,...,earthquake,7.548,7.383,0.019,0.188,0.087,0.006,0.004,1,1
3,2020-12-27 15:39:16.359999,34.17,22.09,121.32,3.94,ml,152,LAY,TW,0,...,earthquake,2.078,0.461,0.017,0.167,0.150,0.005,0.030,1,1
4,2020-12-27 15:39:16.359999,34.17,22.09,121.32,3.94,ml,152,LYUB,TW,10,...,earthquake,22.892,24.627,0.384,3.762,1.739,0.092,0.049,1,1
